# 测试TFRecord文件是否可用

In [1]:
import tensorflow as tf
import numpy as np
import cv2


filename_queue = tf.train.string_input_producer(['E:\\friedhelm\\face_detection_VGG\\face_test_224.tfrecords'],shuffle=True,num_epochs=1)

reader = tf.TFRecordReader()
_, serialized_example = reader.read(filename_queue) #返回文件名和文件

features = tf.parse_single_example(serialized_example,
                               features={
                               'label':tf.FixedLenFeature([],tf.int64),
                               'img':tf.FixedLenFeature([],tf.string),
                               })
img=tf.decode_raw(features['img'],tf.uint8)
label=tf.cast(features['label'],tf.int32)
img = tf.reshape(img, [224,224,3])   
#     img=img_preprocess(img)
min_after_dequeue = 10000
batch_size = 64
capacity = min_after_dequeue + 10 * batch_size
image_batch, label_batch = tf.train.shuffle_batch([img, label], 
                                                    batch_size=batch_size, 
                                                    capacity=capacity, 
                                                    min_after_dequeue=min_after_dequeue,
                                                    num_threads=7)  

i=0
with tf.Session() as sess:
    sess.run((tf.global_variables_initializer(),
              tf.local_variables_initializer()))
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess,coord=coord)    
    while(1):
        i=i+1
        if(i%9==1):
            print(sess.run(label_batch))

[[[180 157 135]
  [181 158 136]
  [181 158 136]
  ...
  [ 55  23  10]
  [ 69  36  21]
  [ 77  45  26]]

 [[181 158 136]
  [181 158 136]
  [181 158 136]
  ...
  [ 57  25  12]
  [ 75  45  28]
  [ 87  55  36]]

 [[181 158 136]
  [181 158 136]
  [182 159 137]
  ...
  [ 64  35  21]
  [ 94  64  47]
  [110  78  59]]

 ...

 [[ 32  68  68]
  [ 35  71  71]
  [ 46  80  80]
  ...
  [ 51  28  20]
  [ 50  27  19]
  [ 51  28  20]]

 [[ 22  58  58]
  [ 25  61  61]
  [ 38  72  72]
  ...
  [ 52  29  21]
  [ 51  28  20]
  [ 52  30  19]]

 [[ 19  56  54]
  [ 22  59  57]
  [ 36  70  69]
  ...
  [ 53  30  22]
  [ 52  30  19]
  [ 52  30  19]]]
INFO:tensorflow:Error reported to Coordinator: <class 'tensorflow.python.framework.errors_impl.CancelledError'>, Enqueue operation was cancelled
	 [[Node: shuffle_batch/random_shuffle_queue_enqueue = QueueEnqueueV2[Tcomponents=[DT_UINT8, DT_INT32], timeout_ms=-1, _device="/job:localhost/replica:0/task:0/device:CPU:0"](shuffle_batch/random_shuffle_queue, Reshape/_27, C